In [4]:
import numpy as np
import cv2

def run_main(video_path):
    cap = cv2.VideoCapture(video_path)
    # 영상 파일이 정상적으로 열렸는지 확인
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # 프레임 크기를 설정합니다 (영상 파일 자체 크기를 그대로 사용할 수 있음)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 3000)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 3000)

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # 프레임을 더 이상 읽을 수 없으면 종료

        # ROI 설정
        roi = frame[0:500, 0:500]
        
        # 그레이스케일 변환 및 블러링 처리
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray_blur = cv2.GaussianBlur(gray, (15, 15), 0)
        
        # 적응형 임계값 처리
        thresh = cv2.adaptiveThreshold(
            gray_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV, 11, 1
        )
        
        # 모폴로지 클로징 연산
        kernel = np.ones((3, 3), np.uint8)
        closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=4)
        
        # 윤곽선 찾기
        contours, hierarchy = cv2.findContours(closing.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area < 2000 or area > 4000:
                continue
            if len(cnt) < 5:
                continue
            # 윤곽선을 따라 타원 맞추기
            ellipse = cv2.fitEllipse(cnt)
            cv2.ellipse(roi, ellipse, (0, 255, 0), 2)
        
        # 영상 표시
        cv2.imshow("Morphological Closing", closing)
        cv2.imshow("Adaptive Thresholding", thresh)
        cv2.imshow('Contours', roi)

        # 'q'를 누르면 루프 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    run_main('coin2.mp4')  # 여기에 영상 파일 경로 입력